In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import sys
sys.path.append('../')
import src.cleaning_functions as cf

In [2]:
load_dotenv()

True

In [3]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

In [6]:
cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

In [ ]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Librería"
}